LangChainMemory
```
이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3x 부터는 LCEL 기반으로 체인을 구성, RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [8]:
# 환경설정
%pip install --quiet langchain-openai langchain-community python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory # 인메모리 -> 어플리케이션 자체에 저장, 외부 불가능 #블로그 
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")


human: 안녕하세요 제 이름은 홍길동 입니다.
ai: 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [9]:
# # Radis 기반  채팅 기록 저장소
# from langchain_redis import RedisChatMessageHistory
# import os
# REDIS_URL =os.getenv("REDIS_URL","redis//localhost:6379") # 환경변수에서 REDIS_URL을 가져오고, 없으면 기본값 사용
# session_id = "user_123" # 세션 ID 설정
# history = RedisChatMessageHistory(
#     session_id=session_id,
#     redis_url=REDIS_URL)
# history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
# history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# # 현재까지의 대화 내용 확인
# for msg in history.messages:
#     print(f"{msg.type}: {msg.content}")


<function os.getenv(key, default=None)>

In [25]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중 사용자 챗봇
# 핵심 : session_id를 키로하는 메모리 저장소 만들고 사용자의 대환느 키별로 저장한다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다 질문에 친절하고 자세히 답변해주세요."),
     # history 키로 전달된 메세지 목록은 체인 실행 시 해당 위치에 넣겠다는 의미 
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)



In [26]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [27]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현

In [28]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 전체 매핑 
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """세션 ID에 해당하는 대화 기록을 반환합니다. 존재하지 않으면 새로 생성합니다."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [29]:
# 두개의 세션을 번갈아가며 대화 RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다. 
sessions = ['user_a', 'user_b']
questions = [
    "안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?", # user_a의 첫번째 질문
    "저는 김철수입니다. 당신은 어떤 일을 하시나요?", # user_b의 첫번째 질문
    "저는 프로그래머입니다. 당신은 어떤 일을 하시나요?",# user_a의 두번째 질문
    "저는 디자이너입니다. 요즘 어떤 프로젝트를 하고 계신가요?" # user_b의 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question},config={'configurable': {'session_id': session_id}})
    print(f"Session {session_id} 질문: {question}")
    print(f"Session {session_id} 챗봇: {result}\n")


16:37:39 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_a 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
Session user_a 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

16:37:40 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_b 질문: 저는 김철수입니다. 당신은 어떤 일을 하시나요?
Session user_b 챗봇: 안녕하세요, 김철수님! 저는 여러분의 질문에 답변하고, 정보 제공, 상담, 그리고 다양한 주제에 대해 대화하는 역할을 하는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

16:37:43 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_a 질문: 저는 프로그래머입니다. 당신은 어떤 일을 하시나요?
Session user_a 챗봇: 저는 다양한 질문에 답변하고 정보를 제공하는 역할을 하고 있습니다. 프로그래밍, 기술, 과학, 문화 등 여러 분야에 대한 질문에 도움을 드릴 수 있습니다. 프로그래밍 관련해서 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!

16:37:46 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_b 질문: 저는 디자이너입니다. 요즘 어떤 프로젝트를 하고 계신가요?
Ses

In [30]:
result = chatbot.invoke({'input': "저는 철수에요, 반갑습니다"},config={'configurable': {'session_id': 'user_c'}})
print(f"Session user_c 질문: 저는 철수에요, 반갑습니다")
print(f"Session user_c 챗봇: {result}\n")

16:40:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_c 질문: 저는 철수에요, 반갑습니다
Session user_c 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 것이 있다면 말씀해 주세요.



In [31]:
result = chatbot.invoke({'input': "저는 누구라고요?"},config={'configurable': {'session_id': 'user_a'}})
result

16:42:59 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'홍길동님이라고 말씀하셨습니다! 혹시 더 알고 싶으신 내용이나 다른 질문이 있으신가요?'

In [32]:
result = chatbot.invoke({'input': "저는 누구라고요?"},config={'configurable': {'session_id': 'user_b'}})
result

16:43:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'김철수님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 질문이 있으신가요? 언제든지 말씀해 주세요!'

In [33]:
result = chatbot.invoke({'input': "저는 누구라고요?"},config={'configurable': {'session_id': 'user_c'}})
result

16:43:26 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'철수님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 이야기를 나누고 싶으신가요?'

요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버젼에서는 직접 요약용 체인을 만들어서 chatMessageHistory
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
- 요약결과를 메모리에 시스템 메세지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력시 요약된 맥락 + 최근 몇 메시지만 참고해서 llm 전달
```

In [34]:
# 요약용 프롬프트 탬플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요."),
    ("human", "{conversation}"), # 전체 대화내용을 하나의 문자열로 전달
])
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [36]:
store.keys() # 현재 저장된 세션 ID 확인

dict_keys(['user_a', 'user_b', 'user_c'])

In [37]:
# user_d 세션에 대화내용을 기록 긴 대화 생성
long_quries = [
    "안녕, 우리 오늘 뭐하려고 했지?",
    "아 맞다 내일 회의자료 준비해야지, 회의는 몇시지지?",
    "그 회의에 누가 참석하는지 기억나니니?",
    "단위프로젝트 진행 상황도 공유해야 할끼?",
    "최근에 이야기 했던 새로운 기능에대한 업데이트는 있어?",
]
session_id = 'user_d'
for q in long_quries:
    answer = chatbot.invoke({'input': q},config={'configurable': {'session_id': session_id}})

print(f'요약전 user_d의 메모리 메시지 개수 : {len(store[session_id].messages)}')
print(store[session_id]) # 요약하기전 대화 내용 

17:25:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:19 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
요약전 user_d의 메모리 메시지 개수 : 10
Human: 안녕, 우리 오늘 뭐하려고 했지?
AI: 안녕하세요! 오늘 어떤 주제에 대해 이야기하고 싶으신가요? 궁금한 점이나 나누고 싶은 이야기가 있다면 말씀해 주세요. 함께 이야기해보아요!
Human: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지지?
AI: 회의 시간이 기억나지 않으신가요? 회의 일정은 보통 이메일이나 캘린더에 기록되어 있을 텐데요. 확인해보시고, 만약 찾기 어렵다면 회의 주최자에게 직접 문의해보시는 것도 좋은 방법입니다. 회의 준비에 도움이 필요하시면 말씀해 주세요! 어떤 자료를 준비해야 하는지에 대해서도 도와드릴 수 있습니다.
Human: 그 회의에 누가 참석하는지 기억나니니?
AI: 회의 참석자에 대한 정보는 제가 알 수 없지만, 보통 회의 초대 이메일이나 캘린더 초대장에 참석자 목록이 포함되어 있습니다. 그곳에서 확인해보시면 좋을 것 같아요. 만약 참석자 목록을 정리해야 하거나, 회의

In [38]:
# 전체대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원보 유지
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
# 요약 대상 대화내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

if len(message) > 2:
    original_dialog = '\n'.join([f'{msg.type.upper()},{msg.content}' for msg in message[:-2]])  # 마지막 QA 쌍 제외
else:
    original_dialog = '\n'.join([f'{msg.type.upper()},{msg.content}' for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({'conversation': original_dialog})
print("== 요약 결과 ==")
print(summary_text)
# 기존 메모리를 요약으로 교체 : 이전내용 요약본 + 최근 QA유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f"요약:{summary_text}"))
# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2:
    last_user_message = message[-2]
    last_ai_message = message[-1]
    if isinstance(last_user_message, HumanMessage):
        new_history.add_user_message(last_user_message.content)
    else:
        new_history.messages.append(last_user_message)
    if isinstance(last_ai_message, AIMessage):
        new_history.add_ai_message(last_ai_message.content)
    else:
        new_history.messages.append(last_ai_message)

# 메모리 교체
store[session_id] = new_history

17:48:37 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
== 요약 결과 ==
대화 요약:

HUMAN은 내일 회의 자료를 준비해야 한다고 언급하며 회의 시간을 확인하려고 했다. AI는 회의 일정이 이메일이나 캘린더에 기록되어 있을 것이라고 안내하고, 참석자 목록도 그곳에서 확인할 수 있다고 설명했다. HUMAN은 단위 프로젝트의 진행 상황을 공유해야 하는지 물었고, AI는 진행 상황 공유의 중요성을 강조하며 포함해야 할 내용(프로젝트 목표, 진행 상황, 문제점 및 해결 방안, 다음 단계)을 제안했다.


In [39]:
print(len(store[session_id].messages))  # 요약 후 대화 내용 확인

3
